# Recherche Predictive - Vision 2026 (LightGBM)


In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
import joblib
import os

In [2]:
# Chargement et Feature Engineering (Standardise)
df_adm = pd.read_csv('../data/raw/admissions_hopital_pitie_2024.csv')
df_adm['date_entree'] = pd.to_datetime(df_adm['date_entree'])
daily_data = df_adm.groupby('date_entree').size().rename('admissions').reset_index()
daily_data = daily_data.set_index('date_entree').asfreq('D', fill_value=0)

def create_features(df):
    df = df.copy()
    df['month_sin'] = np.sin(2 * np.pi * df.index.month / 12)
    df['month_cos'] = np.cos(2 * np.pi * df.index.month / 12)
    df['day_sin'] = np.sin(2 * np.pi * df.index.dayofweek / 7)
    df['day_cos'] = np.cos(2 * np.pi * df.index.dayofweek / 7)
    
    holidays = ['2024-01-01', '2024-04-01', '2024-05-01', '2024-05-08', 
                '2024-05-09', '2024-05-20', '2024-07-14', '2024-08-15', 
                '2024-11-01', '2024-11-11', '2024-12-25']
    holiday_dates = pd.to_datetime(holidays)
    df['is_holiday'] = df.index.strftime('%Y-%m-%d').isin(holidays).astype(int)
    df['days_to_holiday'] = [(holiday_dates[holiday_dates >= d].min() - d).days if any(holiday_dates >= d) else 365 for d in df.index]
    
    df['dayofyear_sin'] = np.sin(2 * np.pi * df.index.dayofyear / 365)
    df['dayofyear_cos'] = np.cos(2 * np.pi * df.index.dayofyear / 365)
    df['weekofyear'] = df.index.isocalendar().week.astype(int)
    df['dayofmonth'] = df.index.day
    
    df['lag1'] = df['admissions'].shift(1)
    df['lag2'] = df['admissions'].shift(2)
    df['lag7'] = df['admissions'].shift(7)
    df['lag14'] = df['admissions'].shift(14)
    
    df['roll_mean_3'] = df['admissions'].shift(1).rolling(window=3).mean()
    df['roll_mean_7'] = df['admissions'].shift(1).rolling(window=7).mean()
    df['roll_max_7'] = df['admissions'].shift(1).rolling(window=7).max()
    df['roll_min_7'] = df['admissions'].shift(1).rolling(window=7).min()
    df['roll_std_7'] = df['admissions'].shift(1).rolling(window=7).std()
    
    return df.dropna()

features_df = create_features(daily_data)
FEATURES = ['month_sin', 'month_cos', 'day_sin', 'day_cos', 'is_holiday', 'days_to_holiday', 
            'dayofyear_sin', 'dayofyear_cos', 'weekofyear', 'dayofmonth', 'lag1', 'lag2', 'lag7', 'lag14', 
            'roll_mean_3', 'roll_mean_7', 'roll_max_7', 'roll_min_7', 'roll_std_7']
TARGET = 'admissions'

train_df = features_df.iloc[:-30]
test_df = features_df.iloc[-30:]
X_train, y_train = train_df[FEATURES], train_df[TARGET]
X_test, y_test = test_df[FEATURES], test_df[TARGET]

## GridSearch avec LightGBM

In [ ]:
param_grid = {
    'n_estimators': [1000, 2000],
    'learning_rate': [0.01, 0.05],
    'num_leaves': [31, 64, 128],
    'feature_fraction': [0.8, 0.9],
    'bagging_fraction': [0.8, 0.9],
    'bagging_freq': [5]
}

tscv = TimeSeriesSplit(n_splits=5)
grid = GridSearchCV(
    lgb.LGBMRegressor(objective='regression', random_state=42, verbose=-1), 
    param_grid, 
    cv=tscv, 
    scoring='neg_mean_absolute_error', 
    verbose=1, 
    n_jobs=-1
)

print("Exécution de la Cross-Validation LightGBM...")
grid.fit(X_train, y_train)
best_lgb = grid.best_estimator_

print(f"Meilleurs Hyperparamètres LightGBM : {grid.best_params_}")

Exécution de la Cross-Validation LightGBM...
Fitting 5 folds for each of 48 candidates, totalling 240 fits


## Analyse des Performances

In [ ]:
y_pred = best_lgb.predict(X_test)
mae_val = -grid.best_score_
mae_test = mean_absolute_error(y_test, y_pred)

print(f"MAE moyenne CV (LGBM) : {mae_val:.2f}")
print(f"MAE Final Test (LGBM) : {mae_test:.2f}")

fig = go.Figure()
fig.add_trace(go.Scatter(x=test_df.index, y=y_test, name='Réel (Décembre)', line=dict(color='#005ba1', width=3)))
fig.add_trace(go.Scatter(x=test_df.index, y=y_pred, name='LightGBM Optimisé', line=dict(color='#00a650', dash='dot', width=3)))
fig.update_layout(title='LightGBM : Performance sur Décembre', template='plotly_dark')
fig.show()